In [8]:
import numpy as np
import pandas as pd
import xarray as xr
import os, json
import tensorflow as tf
from scipy.stats import qmc

In [4]:
# get config information
utils_dir = '/glade/u/home/linnia/CLM6-PPE/utils/'

with open(os.path.join(utils_dir, "param_info.json"), "r") as f:
    param_info = json.load(f)

with open(os.path.join(utils_dir, "biome_info.json"), "r") as f:
    biome_info = json.load(f)

universal_param_names = param_info['u_params']
pft_param_names = param_info['pft_params']
n_universal_params = len(param_info['u_params'])
n_pft_params = len(param_info['pft_params']) # Number of parameters per PFT


In [5]:
# load observational data
obs = xr.open_dataset('~/ctsm6_ppe/analysis_lhc/wave3/wave3_obsStatistics_sudokuBiomes.nc')

In [9]:
# create universal sample
n_samples = 1000

sampler = qmc.LatinHypercube(d=n_universal_params)

s = sampler.random(n=n_samples)
universal_parameter_samples = pd.DataFrame(s,columns=universal_param_names)

In [44]:
# stack up all your observational data and emulators for each input metric (variable and biome)
paths = {
    'lai': './emulators/biome_lai_',
}

biomes = [13]
obs_mean = []
obs_var = []
emulators = [] 
biome_list = [] 

for b in biomes:
    biome_list.append(str(b))

    obs_mean.append(obs.LAI_mean.isel(biome=b).values)
    obs_var.append(obs.LAI_stdev.isel(biome=b).values**2)

    emulators.append(tf.saved_model.load(paths['lai']+str(b)))

In [ ]:
n_pfts = 14
n_psamp = 1000
I_tolerance = 3.0 # implausibility tolerance (defined as float)

all_nroy_param_sets_from_universal = [] # List to collect NROY sets from each universal sample

# Loop over each universal parameter sample
#for u_idx in range(len(universal_parameter_samples)):
for u_idx in range(100):
    # create sample (dataframe) for the current universal parameter set
    usample_values = np.tile(universal_parameter_samples.iloc[[u_idx]].values,(n_psamp,1))
    psample_values = np.random.rand(n_psamp,n_pfts*n_pft_params)
    combined_values = np.concatenate([usample_values,psample_values],axis=1)

    all_pft_param_column_names = []
    for pft_id in range(1, n_pfts + 1):
        for param_name in pft_param_names:
            all_pft_param_column_names.append(f"pft{pft_id}_{param_name}")
    all_columns = universal_param_names + all_pft_param_column_names
    sample_df = pd.DataFrame(combined_values, columns=all_columns)


    # History matching for the current universal parameter set
    num_metrics = len(emulators)
    all_implausibilities_for_samples = np.empty((n_psamp, num_metrics))

    for metric_ix in range(num_metrics):
        biome_id = str(biome_list[metric_ix]) # Ensure biome_id is an integer for dict lookup
        emulator_object = emulators[metric_ix] # Corrected: access emulator from a list/array

        # call_emulator expects biome_info as dict
        y_pred, y_pred_var = call_biome_emulator(sample_df, biome_id, emulator_object, biome_info)

        # calc_Implausibility should return a 1D array of implausibility values for all 'n_psamp' samples
        metric_I = calc_Implausibility(y_pred, y_pred_var, obs_mean[metric_ix], obs_var[metric_ix])
        
        # Store the implausibility values for the current biome/metric
        all_implausibilities_for_samples[:, metric_ix] = metric_I.squeeze()

    # identify parameter sets that are Not Ruled Out Yet for the current universal sample
    is_nroy_mask = (all_implausibilities_for_samples < I_tolerance).all(axis=1)
    current_u_nroy_param_sets = sample_df[is_nroy_mask]
    
    # Append the NROY sets found for this universal parameter to the master list
    all_nroy_param_sets_from_universal.append(current_u_nroy_param_sets)

    print(f"For universal sample {u_idx}: Found {len(current_u_nroy_param_sets)} NROY parameter sets out of {n_psamp} samples.")

# Concatenate all NROY_param_sets from all universal samples
NROY_param_sets = pd.concat(all_nroy_param_sets_from_universal).reset_index(drop=True)


For universal sample 0: Found 60 NROY parameter sets out of 1000 samples.
For universal sample 1: Found 918 NROY parameter sets out of 1000 samples.
For universal sample 2: Found 909 NROY parameter sets out of 1000 samples.
For universal sample 3: Found 852 NROY parameter sets out of 1000 samples.
For universal sample 4: Found 994 NROY parameter sets out of 1000 samples.
For universal sample 5: Found 508 NROY parameter sets out of 1000 samples.
For universal sample 6: Found 1000 NROY parameter sets out of 1000 samples.
For universal sample 7: Found 996 NROY parameter sets out of 1000 samples.
For universal sample 8: Found 967 NROY parameter sets out of 1000 samples.
For universal sample 9: Found 997 NROY parameter sets out of 1000 samples.
For universal sample 10: Found 1000 NROY parameter sets out of 1000 samples.
For universal sample 11: Found 890 NROY parameter sets out of 1000 samples.
For universal sample 12: Found 972 NROY parameter sets out of 1000 samples.
For universal sample 